# Prerequisites
Module Needed:
- pyproj (the key module: for coordinate transformation)
- pandas (to process csv file)
- numpy (optional)

you can simply run `pip install` or `conda install` to install the module needed

```bash
pip install pyproj
pip install pandas
pip install numpy
```

In [83]:
from pyproj import CRS
from pyproj import Transformer
import pandas as pd
try:
    import numpy as np
except:
    print("You can install numpy to improve the accuracy of the result.")

# Function

Parameters:

- **header**

    a list of string including the header of the csv file (can only includes the columns you need).
    must include the following labels:
    - longitude
    - latitude
    - altitude

- **input_csv**
- **output_csv**

    the input and output csv file path (support both relative path and absolute path)

- **epsg**: 

    the epsg code of the utm zone (or other projected coordinates) you want to convert to, which can be found [here](https://epsg.io/).
    
    - to identify the utm zone, you can use [this website](https://awsm-tools.com/lat-long-to-utm)

    - to find the epsg code corresponding to the utm zone, you can use [this website](https://epsg.io/)

    default: epsg=32650 (utm zone 50N)

    Area of use: Between 114°E and 120°E, northern hemisphere between equator and 84°N, onshore and offshore

    Brunei. China. Hong Kong. Indonesia. Malaysia - East Malaysia - Sarawak. Mongolia. Philippines. Russian Federation. Taiwan.

- **flag**
    
    whether to use numpy to improve the accuracy using float64 (default: False)

- **base**

    a size 2 list of number representing the base coordinate in RTK (default: None)

    base[0] should be the longitude of the base coordinate, base[1] should be the latitude of the base coordinate

    if base is None, the base coordinate will be the first coordinate in the csv file

    if base is not None, the base coordinate will be the coordinate you set

    e.g. base = [114.123456, 22.123456]

In [84]:
def rtk2xyz(header, input_csv, output_csv, epsg = 32650, flag = False, base = []):
    """[RTK].csv to [XYZ].csv

    Args:
        header (string list): header of the csv file (can only includes the columns you need)
        input_csv (string): path of RTK csv file
        output_csv (string): path of XYZ csv file
        epsg (int): epsg code of the utm zone or other projected coordinate (default: 32650 which is the code of UTM50N)
        flag (bool): whether to use numpy to improve the accuracy using float64 (default: False)
        base (number list): [longitude, latitude] of the base point (default: [])
    """
    
    rtk_crs = CRS.from_epsg(4326) # WGS84 -- GPS所用坐标系统
    utm_crs = CRS.from_epsg(epsg) # UTM50N -- UTM投影坐标系统（正东方向为x轴正方向，正北方向为y轴正方向）
    transformer = Transformer.from_crs(rtk_crs, utm_crs)

    # 从csv读取
    if(flag):
        data = pd.read_csv(input_csv,usecols=header,dtype={'longitude':np.float64,'latitude':np.float64})
    else:
        data = pd.read_csv(input_csv,usecols=header,dtype={'longitude':float,'latitude':float})

    # 基准点
    if(base==[]):
        base_longitude = data['longitude'][0]
        base_latitude = data['latitude'][0]
        base_x , base_y = transformer.transform(base_latitude,base_longitude)
    else:
        base_x , base_y = transformer.transform(base[1],base[0])

    # 整体运算
    data['longitude'],data['latitude'] = transformer.transform(data['latitude'],data['longitude'])
    data['longitude'] = data['longitude'] - base_x
    data['latitude'] = data['latitude'] - base_y

    data.rename(columns={'longitude':'x','latitude':'y','altitude':'z'},inplace=True)

    # 输出到csv
    data.to_csv(output_csv,index=False)

# Usage

In [85]:
from rtk2xyz import rtk2xyz
rtk2xyz(['%time','field.header.seq','field.header.stamp','longitude','latitude','altitude'],'./RTK17.csv','./XYZ17.csv',flag=True)
rtk2xyz(['%time','field.header.seq','field.header.stamp','longitude','latitude','altitude'],'./RTK18.csv','./XYZ18.csv',flag=True,base=[114.20877497,22.69096604])

&copyright; 2023 [LysanderT](https://github.com/LysanderT/RTK2XYZ)